# SETUP CELLS

In [1]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv
import requests
import json

# === Setup Paths ===
sys.path.insert(0, os.path.abspath("./genie-worksheets/src"))
root_dir = os.getcwd()

# === Load Environment ===
env_file = os.path.join(root_dir, ".env")
load_dotenv(env_file)

# === Verify API Key ===
AGENT_KEY = os.getenv("LLM_API_KEY")
BASE_URL = os.getenv("LLM_API_BASE_URL")
DATABASE_KEY = os.getenv("EXA_API_KEY")

# print(AGENT_KEY)


if not AGENT_KEY or AGENT_KEY == "your_api_key_here":
    raise ValueError("❌ Please set your LLM_API_KEY in .env file")

print("✓ Environment loaded")
print(f"✓ API Base URL: {BASE_URL}")


# === Import only what you need from Genie ===
from worksheets.specification.from_spreadsheet import gsheet_to_classes
print("✓ Genie Worksheets imported")

✓ Environment loaded
✓ API Base URL: https://api.openai.com/v1
✓ Genie Worksheets imported


In [2]:
# Ensure Environment is actually working
import requests
import json
from dotenv import load_dotenv

response = requests.post(
    f"{BASE_URL}/chat/completions",
    headers={
        "Authorization": f"Bearer {AGENT_KEY}",
        "Content-Type": "application/json"
    },
    json={
        "model": "gpt-4o-mini",
        "messages": [
            {"role": "user", "content": "Say hi in one word"}
        ],
        "max_tokens": 5
    }
)

if response.status_code == 200:
    result = response.json()
    print("Success!")
    print(f"Response: {result['choices'][0]['message']['content']}")
    print(f"Tokens used: {result.get('usage', {}).get('total_tokens', 'N/A')}")
else:
    print(f"Error {response.status_code}: {response.text}")

Success!
Response: Hello!
Tokens used: 14


In [3]:
current_dir = os.getcwd()
root_dir = os.path.join(current_dir, "genie-worksheets")

# Credentials for reading the spreadsheet
!curl -L -o creds.zip "https://drive.google.com/uc?export=download&id=11QvSs2JZ5qpPrCvbX66Dg8pxfM_B8GaH"
!unzip creds.zip -d genie-worksheets/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  3009  100  3009    0     0   1863      0  0:00:01  0:00:01 --:--:-- 10595
Archive:  creds.zip
  inflating: genie-worksheets/worksheets/token.json  
  inflating: genie-worksheets/worksheets/credentials.json  
  inflating: genie-worksheets/worksheets/service_account.json  


In [4]:
!mv "genie-worksheets/worksheets/credentials.json" "genie-worksheets/src/worksheets/config/credentials.json"
!mv "genie-worksheets/worksheets/service_account.json" "genie-worksheets/src/worksheets/config/service_account.json"
!mv "genie-worksheets/worksheets/token.json" "genie-worksheets/src/worksheets/config/token.json"

creds1 = os.path.join(root_dir, "src", "worksheets", "config", "credentials.json")
creds2 = os.path.join(root_dir, "src", "worksheets", "config", "service_account.json")
creds3 = os.path.join(root_dir, "src", "worksheets", "config", "token.json")

for cred in [creds1, creds2, creds3]:
  assert os.path.exists(cred), f"Cannot find the credential file: {cred}"

In [15]:
from worksheets.agent.config import agent_api

CELESTE_COLLECTIBLES = [
    {
      "collectible_id": "strawberry_11",
      "collectible_type": "strawberry",
      "collectible_number": 12,
      "hidden": True,
      "section": "Crossing",
      "info": [
        {
          "instruction": "[Missable] After the room with the up-arrow sign, head to the top-left of the room and dash through the breakable ice. Here you will find strawberry 10 floating near the left wall. To get to strawberry 11, jump onto the floating platform above and dash into the breakable ice in the middle of the roof - where you will be taken to another room. In the next room: Climb up onto the wall to jump onto the crumbling platform, then to the vertical lift. Use the vertical lift to boost up onto the falling floating ice, then quickly dash to safety. Make your way to the next room which contains strawberry 11. Once you’re in the strawberry room: Dash onto the dirt platform. Fall to grab the strawberry then dash right to collect the crystal, dashing up-right to land safely and collect the strawberry.",
          "source": "STEAM COMMUNITY"
        },
        {
          "instruction": "In the same room as Strawberry #10, climb the right-hand wall to reach the sole platform. Jump out into the middle and dash upward to find a new screen. Climb through this room, using the falling blocks and the geared platform to make it to the top. In the following room, you’ll spot the strawberry. Dash and climb your way to the brown platform. Drop down to the strawberry and dash in between the spikes to collect the green crystal. With your dash restored, dash diagonally upward to safety.",
          "source": "IGN"
        },
        {
          "instruction": "But wait, we’re not done here yet! Jump and grab onto the wall on the right, then jump and dash onto the floating island. Can you see the crumbling part in the wall? Dash up into it and proceed to the next area. There are no strawberries here so proceed to the top right and through to the next frame. There’s your strawberry! Fall down, dash up-right, grab onto the dry wall and climb up it. Fall off the wooden platform to the right, collect the strawberry and dash between the spikes into the diamond. Dash again up-right past the spikes and there we go!",
          "source": "SWITCHABOO"
        }
      ]
    }
    # Will eventually be a full database, more will be added later.
]

@agent_api("get_hint", "Get a hint for a Celeste level")
def get_hint(
    collectable_id: str, 
    hint_tier: int
):
    """
    Get a hint for a specific level.
    
    Args:
        level_id: The level ID
        room_id: The room ID
        hint_tier: Which hint to return (0, 1, or 2)
    """
    # Find the level
    level = next(
        (lvl for lvl in CELESTE_COLLECTIBLES
         if lvl["collectible_id"] == "strawberry_11"),
        None
    )
    
    # Validate hint_tier
    if hint_tier < 0 or hint_tier >= len(level["hint_tiers"]):
        return {
            "status": "error",
            "message": f"Hint tier {hint_tier} not available. Available: 0-{len(level['hint_tiers'])-1}"
        }
    
    return {
        "status": "success",
        "collectable_id": collectable_id,
        "hidden": level["hidden"],
        "info": level["info"],
        "hint_tier": hint_tier,
        "hint": level["hint_tiers"][hint_tier]
    }

In [17]:
# Enter the ID of your google sheet here
gsheet_id_default = "1kpbfZkxR288BmRQ7oxaIbGfPrP4xgqDrAcx8OQz6qcg"

Ensure to share with the following account: ``yelpbotvm@mixedinitiative.iam.gserviceaccount.com``

In [ ]:
botname = "CelesteCollectibleGenie"
starting_prompt = "Hello! I am CelesteCollectibleGenie. I can help you with hints and tips for collecting items in Celeste levels. How many strawberries have you collected so far? Check the pause menu in the top-left corner of your game to see."
description = "CelesteCollectibleGenie is an assistant that provides hints and tips for players trying to collect items in the game Celeste. Users can specify which collectible they are aiming for, and CelesteCollectibleGenie will offer tiered hints to help them progress. The assistant is knowledgeable about various mechanics in the game, such as dashing and jumping, and can provide strategic advice based on the player's current situation. CelesteCollectibleGenie aims to enhance the gaming experience by offering helpful guidance without giving away complete solutions, encouraging players to think critically and improve their skills."

In [19]:
# Helper code to convert the dialogue state to JSON format

from worksheets.utils.annotation import get_agent_action_schemas, get_context_schema
from worksheets.core.dialogue import CurrentDialogueTurn

def convert_to_json(dialogue: list[CurrentDialogueTurn]):
    json_dialogue = []
    for turn in dialogue:
        json_turn = {
            "user": turn.user_utterance,
            "bot": turn.system_response,
            "turn_context": get_context_schema(turn.context),
            "global_context": get_context_schema(turn.global_context),
            "system_action": get_agent_action_schemas(turn.system_action),
            "user_target_sp": turn.user_target_sp,
            "user_target": turn.user_target,
            "user_target_suql": turn.user_target_suql,
        }
        json_dialogue.append(json_turn)
    return json_dialogue

In [20]:
# You need to define the models to use for each componenet of Genie Worksheets.
# You can also load the configurations from a yaml file.

from worksheets.agent.config import Config, AzureModelConfig, OpenAIModelConfig

config = Config(
    semantic_parser = OpenAIModelConfig(
        model_name="gpt-4o-mini",
    ),
    response_generator = OpenAIModelConfig(
        model_name="gpt-4o-mini",
    ),
    knowledge_parser=OpenAIModelConfig(),
    knowledge_base=OpenAIModelConfig(),
    validate_response= False,

    prompt_log_path = "logs.log",
    conversation_log_path= "conv_log.json",
)

In [21]:
# Initialize your llm by providing it the prompts directory and your `.env` file which contains the secrets

from worksheets.llm.prompts import init_llm

# Use paths relative to current directory
prompts_path = os.path.join(os.getcwd(), "genie-worksheets", "src", "worksheets", "prompts")
env_path = os.path.join(os.getcwd(), ".env")

init_llm(prompts_path, env_path)

In [26]:
# Main agent builder that generates the agent for you
from worksheets import AgentBuilder, conversation_loop
import json
from loguru import logger

logger.remove()
logger.add(sys.stderr, level="ERROR")

agent_builder = (
    AgentBuilder(
        name=botname,
        description=description,
        starting_prompt=starting_prompt
    )
    .with_gsheet_specification(gsheet_id_default)
)

agent = agent_builder.build(config)


In [23]:
for ws in agent.runtime.genie_worksheets:
  print(ws)

RequestHint(collectible_id: str, hint_tier: Enum['1', '2', '3'], confirm_details: bool)


# INTERACTION WITH AGENT

In [27]:
with agent:
  await conversation_loop(agent, debug=True)

Agent: Hello! I am CelesteCollectibleGenie. I can help you with hints and tips for collecting items in Celeste levels. How many strawberries have you collected so far? Check the pause menu in the top-left corner to see.

request_hint = RequestHint(collectible_id='strawberry_11', hint_tier='1', confirm_details=True)
User: strawberry_11
Agent: > Would you like to proceed with the provided game information for the collectible **strawberry_11**?



2025-12-01 15:55:54.334 | ERROR    | worksheets.core.runtime:execute:311 - Error: get_hint() missing 1 required positional argument: 'hint_tier'
2025-12-01 15:55:54.334 | ERROR    | worksheets.core.runtime:execute:312 - Code: __return = []
hint_message = get_hint(_obj.collectible_id.value, _obj.hint_tier.value)
__return.append(say(hint_message))


request_hint.confirm_details = True
User: strawberry_11
Agent: > Would you like to proceed with the provided game information for the collectible **strawberry_11**?
User: yes
Agent: > Since you confirmed, here’s your hint for the collectible **strawberry_11** at tier 1: 
>
> Look for areas where you can use your dash effectively to reach higher platforms. Pay attention to the environment for any hidden paths or tricky jumps that might lead you to the collectible. Good luck!

